In [1]:
import duckdb

# Conectar ao banco de dados (ou criar um novo)
conn = duckdb.connect('embeddings.db')

In [2]:
# verificando quais extensões já estão instaladas
extensoes_duckdb = conn.execute("""SELECT extension_name, installed, description
                FROM duckdb_extensions();""").fetchdf()
extensoes_duckdb

,extension_name,installed,description
0,arrow,False,A zero-copy data integration between Apache Ar...
1,autocomplete,False,Adds support for autocomplete in the shell
2,aws,False,Provides features that depend on the AWS SDK
3,azure,False,Adds a filesystem abstraction for Azure blob s...
4,delta,False,Adds support for Delta Lake
5,excel,False,Adds support for Excel-like format strings
6,fts,True,Adds support for Full-Text Search Indexes
7,httpfs,False,Adds support for reading and writing files ove...
8,iceberg,False,Adds support for Apache Iceberg
9,icu,True,Adds support for time zones and collations usi...


In [3]:
# Caso ainda não esteja instalado, instala extensão para similarity search
if extensoes_duckdb.query('extension_name == "vss"')['installed'].values[0] == False:
    conn.execute("""INSTALL vss;""")

## carrega o vss
conn.execute("""LOAD vss;""")
# seta a variavél experimental para o vss persistido em memória. 
# Por enquanto não está disponível normalmente na versão 1.0.0
conn.execute("""SET hnsw_enable_experimental_persistence = True""")

In [4]:
# Criar uma tabela para armazenar vetores
conn.execute("""CREATE TABLE embeddings (vec FLOAT[3]);""")
conn.execute("""INSERT INTO embeddings 
             SELECT array_value(a, b, c) 
             FROM range(1, 10) ra(a), range(1, 10) rb(b), range(1, 10) rc(c);
    """)

# cria o index HNSW
conn.execute("""CREATE INDEX my_hnsw_index ON embeddings USING HNSW (vec);""")

# Realizando a busca
conn.execute("""SELECT * 
                FROM embeddings 
                ORDER BY array_distance(vec, [1, 2, 3]::FLOAT[3]) 
                LIMIT 5;""").fetch_df()

,vec
0,"[1.0, 2.0, 3.0]"
1,"[1.0, 3.0, 3.0]"
2,"[1.0, 2.0, 4.0]"
3,"[1.0, 1.0, 3.0]"
4,"[2.0, 2.0, 3.0]"


In [5]:
# por padrão, o index HNSW será criado utiizando a distancia euclidiana l2sq (L2-norm squared).
# para trocar a distância, pode ser utilizado:
conn.execute("""CREATE INDEX my_hnsw_cosine_index
                ON embeddings
                USING HNSW (vec)
                WITH (metric = 'cosine');""")

# as opções são l2sq, cosine e ip (inner product)

In [6]:
conn.execute("""EXPLAIN SELECT * FROM embeddings ORDER BY array_distance(vec, [1, 2, 3]::FLOAT[3]) LIMIT 5;""").fetch_arrow_table()

pyarrow.Table
explain_key: string
explain_value: string
----
explain_key: [["physical_plan"]]
explain_value: [["┌───────────────────────────┐
│         PROJECTION        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             #0            │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│         PROJECTION        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│            vec            │
│            NULL           │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│      HNSW_INDEX_SCAN      │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│embeddings (HNSW INDEX SCAN│
│      : my_hnsw_index)     │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│            vec            │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           EC: 5           │
└───────────────────────────┘                             
"]]